Create 3-d climatology of N and Si boundary values from LiveOcean-based BCs to replace data-based climatology that was only 2d (time and depth, no variation across Strait)

In [1]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import glob
import datetime as dt
import pandas as pd
from matplotlib.colors import LogNorm
%matplotlib inline

In [2]:
with nc.Dataset('/ocean/eolson/MEOPAR/NEMO-forcing/grid/mesh_mask201702_noLPE.nc') as m:
    z=m.variables['gdept_1d'][0,:]

In [3]:
ffmt='m%md%d'

In [4]:
meanSi=np.zeros((365,40,1,95))
stdSi=np.zeros((365,40,1,95))
meanN=np.zeros((365,40,1,95))
stdN=np.zeros((365,40,1,95))
RMSEM=np.zeros((365,40,1,95))
RMSEP=np.zeros((365,40,1,95))
for ii in range(0,365):
    Sis=list()
    Ts=list()
    Ss=list()
    Ns=list()
    idt=dt.datetime(2015,1,1)+dt.timedelta(days=ii)
    stencil='/results/forcing/LiveOcean/boundary_conditions/LiveOcean_v201905*{}.nc'
    flist=glob.glob(stencil.format(idt.strftime(ffmt)))
    for iff in flist:
        with nc.Dataset(iff) as f:
            Sis.append(f.variables['Si'][:,:,:,:95])
            Ts.append(f.variables['votemper'][:,:,:,:95])
            Ss.append(f.variables['vosaline'][:,:,:,:95])
            Ns.append(f.variables['NO3'][:,:,:,:95])
    Sis=np.concatenate(Sis,0)
    Ns=np.concatenate(Ns,0)
    Ts=np.concatenate(Ts,0)
    Ss=np.concatenate(Ss,0)
    meanSi[ii,:,:,:]=np.mean(Sis,0)
    stdSi[ii,:,:,:]=np.std(Sis,0)
    meanN[ii,:,:,:]=np.mean(Ns,0)
    stdN[ii,:,:,:]=np.std(Ns,0)

In [5]:
np.save('/data/eolson/results/MEOPAR/SS36runs/calcFiles/OBC/nmat.npy',meanN)
np.save('/data/eolson/results/MEOPAR/SS36runs/calcFiles/OBC/simat.npy',meanSi)

In [6]:
testN=np.load('/data/eolson/results/MEOPAR/SS36runs/calcFiles/OBC/nmat.npy')
testSi=np.load('/data/eolson/results/MEOPAR/SS36runs/calcFiles/OBC/simat.npy')

In [7]:
np.max(np.abs(testSi-meanSi))

0.0

In [8]:
np.shape(testN)

(365, 40, 1, 95)

In [17]:
temp=np.tile(testN[0,:,:,:],(1,1,1,10))

In [18]:
np.shape(temp)

(1, 40, 1, 950)

In [3]:
f=nc.Dataset('/ocean/eolson/MEOPAR/NEMO-3.6-inputs/boundary_conditions/bioOBC_LOBase.nc')

In [4]:
f

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): deptht(40), yb(1), xbT(870), time_counter(24)
    variables(dimensions): float32 deptht(deptht), float32 nav_lat(yb,xbT), float32 nav_lon(yb,xbT), int32 nbidta(yb,xbT), int32 nbjdta(yb,xbT), int32 nbrdta(yb,xbT), float32 time_counter(time_counter), float32 NO3(time_counter,deptht,yb,xbT), float32 Si(time_counter,deptht,yb,xbT)
    groups: 

In [5]:
test=nc.Dataset('/results/forcing/LiveOcean/boundary_conditions/LiveOcean_v201905_y2015m02d08.nc')

In [7]:
test

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    acknowledgements: Live Ocean http://faculty.washington.edu/pmacc/LO/LiveOcean.html
    creator_email: sallen@eoas.ubc.ca
    creator_name: Salish Sea MEOPAR Project Contributors
    creator_url: https://salishsea-meopar-docs.readthedocs.org/
    institution: UBC EOAS
    institution_fullname: Earth, Ocean & Atmospheric Sciences, University of British Columbia
    summary: Temperature, Salinity, Nitrate, Oxygen, DIC and TALKfrom the Live Ocean model interpolated in space onto the Salish Sea NEMO Model western open boundary. Silicon from Nitrate.
    source: http://nbviewer.jupyter.org/urls/bitbucket.org/salishsea/.../LiveOceanNew
    history: [2019-06-07] File creation.
    dimensions(sizes): time_counter(1), deptht(40), yb(1), xbT(950)
    variables(dimensions): int64 time_counter(time_counter), float64 deptht(deptht), int64 yb(yb), int64 xbT(xbT), float64 vosaline(time_counter,deptht,yb,xbT), flo

In [14]:
f2=nc.Dataset('/ocean/eolson/MEOPAR/NEMO-3.6-inputs/boundary_conditions/bioOBC_LOBase1905.nc','w')

In [15]:
for name, dimension in test.dimensions.items():
        f2.createDimension(name, (len(dimension) if not dimension.isunlimited() else None))

In [16]:
for name, variable in test.variables.items():
        if name not in ('vosaline', 'votemper', 'OXY', 'DIC', 'TA'):
            x = f2.createVariable(name, variable.datatype, variable.dimensions)
            f2[name][:] = 0.0

In [17]:
f2.close()